<a href="https://colab.research.google.com/github/veniaibinder/DS-Unit-1-Sprint-2-Statistics/blob/master/module2/LS_DS_122_Chi2_Tests_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Assignment - Practice Chi^2 Tests

Use the following dataset relating to math scores of students in two different Portugese schools:

<https://archive.ics.uci.edu/ml/datasets/Student+Performance>

### 1) Load the dataset specifically relating to math scores as a new dataframe.

There are two datasets in the `student.zip` file, make sure you use `student-mat.csv`.


In [8]:
# YOUR WORK HERE
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00320/student.zip

--2020-06-16 02:58:47--  https://archive.ics.uci.edu/ml/machine-learning-databases/00320/student.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20478 (20K) [application/x-httpd-php]
Saving to: ‘student.zip.1’

student.zip.1       100%[===================>]  20.00K  --.-KB/s    in 0.06s   

2020-06-16 02:58:48 (311 KB/s) - ‘student.zip.1’ saved [20478/20478]



In [9]:
!unzip student.zip

Archive:  student.zip
replace student-mat.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace student-por.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace student-merge.R? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace student.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [10]:
from scipy import stats
import pandas as pd

df = pd.read_csv('student-mat.csv', sep=';')

print(df.shape)
df.head()


(395, 33)


,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,course,mother,2,2,0,yes,no,no,no,yes,yes,no,no,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,course,father,1,2,0,no,yes,no,no,no,yes,yes,no,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,other,mother,1,2,3,yes,no,yes,no,yes,yes,yes,no,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,home,mother,1,3,0,no,yes,yes,yes,yes,yes,yes,yes,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,home,father,1,2,0,no,yes,yes,no,yes,yes,no,no,4,3,2,1,2,5,4,6,10,10


### 2) Use Chi^2 tests and `stats.chi2_contingency()` to identify:
 - Two pairs of variables that are dependent (are associated with one another).
 - Two pairs of variables that are independent (have no significant relationship).

Does it make intuitive sense why the variables in these pairs might or might not show a relationship to one another? 


In [25]:
#Two pairs of variables that are dependent (are associated with one another).
df['internet'].value_counts().sort_index()
df['studytime'].value_counts().sort_index()
contingency = pd.crosstab(df['internet'], df['studytime'])

contingency

studytime,1,2,3,4
internet,,,,
no,19,37,6,4
yes,86,161,59,23


In [26]:
chi2, p_value, dof, expected = stats.chi2_contingency(contingency)

print("chi2 statistic", chi2)
print("p value", p_value)
print("degrees of freedom",dof)
print("expected frequencies table", expected)

chi2 statistic 3.3831090116205695
p value 0.336241909436802
degrees of freedom 3
expected frequencies table [[ 17.5443038   33.0835443   10.86075949   4.51139241]
 [ 87.4556962  164.9164557   54.13924051  22.48860759]]


In [6]:
# YOUR WORK HERE
#Two pairs of variables that are independent (have no significant relationship).
df['traveltime'].value_counts().sort_index()
df['studytime'].value_counts().sort_index()
contingency = pd.crosstab(df['traveltime'], df['studytime'])

contingency

studytime,1,2,3,4
traveltime,,,,
1,60,133,47,17
2,31,51,16,9
3,11,11,1,0
4,3,3,1,1


In [7]:
chi2, p_value, dof, expected = stats.chi2_contingency(contingency)

print("chi2 statistic", chi2)
print("p value", p_value)
print("degrees of freedom",dof)
print("expected frequencies table", expected)

chi2 statistic 11.200170929144969
p value 0.26223746329595793
degrees of freedom 9
expected frequencies table [[ 68.3164557  128.82531646  42.29113924  17.56708861]
 [ 28.44303797  53.63544304  17.60759494   7.31392405]
 [  6.11392405  11.52911392   3.78481013   1.5721519 ]
 [  2.12658228   4.01012658   1.3164557    0.54683544]]


**Conclusion**

Null Hypothesis: These two variables are "independent" there's no relationship between the two

Alternative Hypothesis: The two variables are "dependent" there is a relationship or association between them.

Based on the chi statistic of 11.2 and p-value of 0.26, we fail to reject the null hypothesis that travel time and study time at home are independent.



### 3) Use NumPy to perform your own chi^2 test "from scratch" 

Pick any of the chi^2 tests that you ran in part 2 and try them on your own. You should get the same results that Scipy got for all four values returned from `chi2_contingency()`

In [0]:
# YOUR WORK HERE

## Stretch goals:

### 1. Refactor your code so it is elegant, readable, and holds reusable code in functions.

In [0]:
# YOUR WORK HERE



### 2. Check For Understanding - Study and write your own explanations/definitions for these topics:

- What is a sample "estimate" in statistics?

- What are hypothesis test? How are they useful?

- What is a "null hypothesis?"

- What is a p-value and what does it represent?

- What does it mean for something to be "statistically significant?"

- What is a test statistic and how does it relate to a p-value?

- What are "degrees of freedom" and how are they calculated in a 1-sample, 2-sample, and chi^2 test for independence?

## Resources

- [Interactive visualize the Chi-Squared test](https://homepage.divms.uiowa.edu/~mbognar/applets/chisq.html)
- [Calculation of Chi-Squared test statistic](https://en.wikipedia.org/wiki/Pearson%27s_chi-squared_test)
- [Visualization of a confidence interval generated by R code](https://commons.wikimedia.org/wiki/File:Confidence-interval.svg)
- [Expected value of a squared standard normal](https://math.stackexchange.com/questions/264061/expected-value-calculation-for-squared-normal-distribution) (it's 1 - which is why the expected value of a Chi-Squared with $n$ degrees of freedom is $n$, as it's the sum of $n$ squared standard normals)